**Importing libraries**

In [16]:
import numpy as np 
import pandas as pd 
import multiprocessing as multi
from scipy.ndimage import gaussian_filter
import matplotlib.pyplot as plt


**Defining constants**

In [17]:
omega_m = 0.272            #matter density paratmeter from Komatsu et al. (2011) 
omega_l = 1 - omega_m      #vacuum density paratmeter assuming flat universe
H_o = 70.4                 #Hubble constant in km s^−1 Mpc^−1 from Komatsu et al. (2011) 
f = omega_m**0.545         #linear velocity growth rate from Tanimura et al. (2020)


**Defining Functions**

In [18]:
def H(z):                           #hubble parameter (in km s^-1 Mpc^-1) using Eq. 4.33 in Peter's book
    return np.sqrt( H_o**2 * ( (1+z)**3 * omega_m + omega_l ) ) 


**Reading galaxies dataset**

In [19]:
%%time
z = 0.42
# data_address = '../input/magneticumsnap027z042-massfiltered/'
data_address = 'Data/'
df_gal = pd.read_csv(data_address + 'massive_galaxies.csv')      #massive_galaxies.csv contains galaxies with mass greater than 1.8 ×10^{11} h^{−1} M_sun as done by Tanimura et al. (2020)
print(df_gal.describe())


            x[kpc/h]       y[kpc/h]       z[kpc/h]     m[Msol/h]
count   93097.000000   93097.000000   93097.000000  9.309700e+04
mean   325576.133087  322500.028738  320613.762863  3.993206e+11
std    184863.273960  182799.587421  183249.915097  3.995129e+11
min         1.864417      10.164207       0.260974  1.800010e+11
25%    163866.950000  166201.380000  162514.560000  2.161480e+11
50%    332449.620000  329260.160000  319925.780000  2.784900e+11
75%    483949.000000  475466.120000  478698.410000  4.192930e+11
max    639996.190000  639980.940000  639993.120000  1.077460e+13
CPU times: user 79.4 ms, sys: 12.5 ms, total: 91.9 ms
Wall time: 91.2 ms


**Reading clusters dataset**

In [45]:
%%time

df_clusters = pd.read_csv(data_address + 'massive_clusters.csv', 
                          usecols = ['x[kpc/h]', 'y[kpc/h]', 'z[kpc/h]', 'm500c[Msol/h]', 'vx[km/s]', 'vy[km/s]', 'vz[km/s]'    ])  #massive_clusters.csv contains clusters with M_500c greater than 10^13.5 h^{-1} M_sun as done by Tanimura et al. (2020)

#removing cluster at edges of the survey volume as done by Tanimura et al. 2020
df_clusters = df_clusters[(df_clusters['x[kpc/h]'] > 120000) & (df_clusters['x[kpc/h]'] < 530000) 
                          & (df_clusters['y[kpc/h]'] > 120000) & (df_clusters['y[kpc/h]'] < 530000) 
                          & (df_clusters['z[kpc/h]'] > 120000) & (df_clusters['z[kpc/h]'] < 530000)]
        
print(df_clusters.describe())

            x[kpc/h]       y[kpc/h]       z[kpc/h]  m500c[Msol/h]  \
count    1592.000000    1592.000000    1592.000000   1.592000e+03   
mean   326225.714152  330893.131420  319484.322167   6.318661e+13   
std    117744.272442  117949.607936  117197.038213   4.845334e+13   
min    120050.600000  120120.120000  120140.480000   3.163600e+13   
25%    225829.552500  234986.600000  221053.682500   3.819940e+13   
50%    328897.030000  336651.920000  313286.765000   4.811530e+13   
75%    427401.647500  433925.950000  426564.770000   7.023110e+13   
max    529929.500000  529911.120000  529783.620000   7.220310e+14   

          vx[km/s]     vy[km/s]     vz[km/s]  
count  1592.000000  1592.000000  1592.000000  
mean     56.881581    47.430418    -7.541318  
std     285.013942   296.601694   277.891742  
min   -1179.530000 -1107.410000 -1064.920000  
25%    -120.642250  -147.272250  -186.943250  
50%      58.529550    60.448700   -13.359600  
75%     238.076250   252.000000   177.276500  
ma

**Specifying prefactors for Eq. 1 of Tanimura et al. (2020)**

In [46]:
a = 1/(1+z)
H(z)
print(a)

pre_fac = (f * a * H(z) / (4 * np.pi))
pre_fac

0.7042253521126761


2.3820143592892085

**Computing density contrast for the Eq. 1**

In [48]:
%%time

pix_size = 5000                      #h^-1 kpc
FWHM = 2000                          #h^-1 kpc, of Gaussian kernel
FWHM_in_pix = 2000/pix_size          #in pixel units
sigma_in_pix = FWHM_in_pix/(2.35482) #in pixel units

def density_field_calc(clus_x, clus_y, clus_z):

    #converting strings into floats
    clus_x = float(clus_x); clus_y = float(clus_y); clus_z = float(clus_z)
    
    #filtering galaxies in a cube of 240,000 h^-1 kpc around the given cluster
    df_gal_select = df_gal[(df_gal['x[kpc/h]'] >= (clus_x - 120000)) & (df_gal['x[kpc/h]'] < (clus_x + 120000))
                            & (df_gal['y[kpc/h]'] >= (clus_y - 120000)) & (df_gal['y[kpc/h]'] < (clus_y + 120000)) 
                             & (df_gal['z[kpc/h]'] >= (clus_z - 120000)) & (df_gal['z[kpc/h]'] < (clus_z + 120000))]
    
    #making copy to extract coordinates of cells containing the galaxies
    df_gal_cube = df_gal_select.copy()
    
    #moving the galxies cube to lie within 0 to 240,000 h^-1 kpc 
    df_gal_cube['x[kpc/h]'] -= (clus_x-120000)
    df_gal_cube['y[kpc/h]'] -= (clus_y-120000)
    df_gal_cube['z[kpc/h]'] -= (clus_z-120000)

    #now divide by 5000 (integer-div) so we get cell coordinates
    df_gal_cube['x[kpc/h]'] = df_gal_cube['x[kpc/h]'] / 5000
    df_gal_cube['y[kpc/h]'] = df_gal_cube['y[kpc/h]'] / 5000
    df_gal_cube['z[kpc/h]'] = df_gal_cube['z[kpc/h]'] / 5000
    
    #making tuples, converting tuples to cell coordinates
    df_gal_cube["cell"] = list(zip(df_gal_cube['x[kpc/h]'].astype(int), df_gal_cube['y[kpc/h]'].astype(int), df_gal_cube['z[kpc/h]'].astype(int)))
    
    #array to store number of galaxies in the cells
    gals_in_cell = np.zeros((48, 48, 48))
    
    #counting number of galaxies in the cells
    for cell in df_gal_cube["cell"]:
        x, y, z = cell
        gals_in_cell[x, y, z] += 1
        
    #determining the overdensity    
    delta = (gals_in_cell/np.mean(gals_in_cell)) - 1
    
#     print(delta[0, :, :])
#     plt.imshow(delta[0, :, :])
#     plt.colorbar()
    
    #smoothing the overdensity
    delta_smooth = gaussian_filter(delta, sigma = sigma_in_pix)
    
#     plt.figure()
#     print(delta_smooth[0, :, :])
#     plt.imshow(delta_smooth[0, :, :])
#     plt.colorbar()
    
    df_gal_select['denom'] = (np.sqrt((df_gal_select['x[kpc/h]'] - clus_x)**2 + (df_gal_select['y[kpc/h]'] - clus_y)**2 
                                   + (df_gal_select['z[kpc/h]'] - clus_z)**2))**3
    print(df_gal_select.describe())
    
#     vx_est = pre_fac * 

clus_coord = [(clus_x, clus_y, clus_z) for clus_x, clus_y, clus_z in zip(df_clusters['x[kpc/h]'], df_clusters['y[kpc/h]'], df_clusters['z[kpc/h]'])]


density_field_calc('416658.59', '455771.69', '72710.742')

# pool = multi.Pool(processes = 42)
# pool.starmap(density_field_calc, clus_coord)

            x[kpc/h]       y[kpc/h]       z[kpc/h]     m[Msol/h]         denom
count    4401.000000    4401.000000    4401.000000  4.401000e+03  4.401000e+03
mean   414052.472079  457187.646033   92557.926067  4.021183e+11  1.573823e+15
std     66866.355939   68213.706705   55060.324866  4.052996e+11  1.228647e+15
min    296816.560000  335774.000000       2.833172  1.800030e+11  0.000000e+00
25%    354711.030000  393400.690000   44557.340000  2.175220e+11  6.022194e+14
50%    411732.310000  459381.380000   91303.219000  2.790660e+11  1.366660e+15
75%    471579.410000  513905.620000  138854.110000  4.285530e+11  2.220936e+15
max    536658.560000  575762.380000  192669.030000  9.539830e+12  7.783121e+15
CPU times: user 62.6 ms, sys: 0 ns, total: 62.6 ms
Wall time: 62.1 ms


/usr/lib/python3/dist-packages/ipykernel_launcher.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
